<a href="https://colab.research.google.com/github/Venu212/Machine-Learning-/blob/master/ConversQueryTAPAS_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [137]:
!git clone https://github.com/google-research/tapas.git


Cloning into 'tapas'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 363 (delta 64), reused 82 (delta 39), pack-reused 236
Receiving objects: 100% (363/363), 334.52 KiB | 606.00 KiB/s, done.
Resolving deltas: 100% (187/187), done.


In [138]:
!ls -l sample_data ./tapas/tapas/utils




ls: cannot access 'sample_data': No such file or directory
./tapas/tapas/utils:
total 348
-rw-r--r-- 1 root root  2734 Oct 30 00:25 beam_runner.py
-rw-r--r-- 1 root root  1232 Oct 30 00:25 constants.py
-rw-r--r-- 1 root root 10495 Oct 30 00:25 experiment_utils.py
-rw-r--r-- 1 root root  2560 Oct 30 00:25 experiment_utils_test.py
-rw-r--r-- 1 root root   982 Oct 30 00:25 file_utils.py
-rw-r--r-- 1 root root  3568 Oct 30 00:25 hparam_utils.py
-rw-r--r-- 1 root root   616 Oct 30 00:25 __init__.py
-rw-r--r-- 1 root root 10055 Oct 30 00:25 interaction_utils_parser.py
-rw-r--r-- 1 root root  8794 Oct 30 00:25 interaction_utils_parser_test.py
-rw-r--r-- 1 root root  4164 Oct 30 00:25 interaction_utils.py
-rw-r--r-- 1 root root  6771 Oct 30 00:25 interaction_utils_test.py
-rw-r--r-- 1 root root  6585 Oct 30 00:25 interpretation_utils.py
-rw-r--r-- 1 root root  4131 Oct 30 00:25 interpretation_utils_test.py
-rw-r--r-- 1 root root  8308 Oct 30 00:25 number_annotation_utils.py
-rw-r--r-- 1 root r

In [139]:
!pip install ./tapas
 

Processing ./tapas
  Using cached https://files.pythonhosted.org/packages/2f/a6/30b0a0bef12283e83e58c1d6e7b5aabc7acfc4110df81a4471655d33e704/cachetools-3.1.1-py2.py3-none-any.whl
  Created wheel for tapas-table-parsing: filename=tapas_table_parsing-0.0.1.dev0-cp36-none-any.whl size=195631 sha256=5d92c6e0095d129478f1807ed1ad60cf0935d0cb94b22db1ffaf22fddbc4010f
  Stored in directory: /tmp/pip-ephem-wheel-cache-olhlzjms/wheels/dd/20/6e/7557deb5d1fc791302cf438ad0522db6e8754a987222cbd66a
Successfully built tapas-table-parsing
ERROR: streamlit 0.70.0 has requirement cachetools>=4.0, but you'll have cachetools 3.1.1 which is incompatible.
ERROR: google-api-python-client 1.7.12 has requirement httplib2<1dev,>=0.17.0, but you'll have httplib2 0.12.0 which is incompatible.
  Found existing installation: tapas-table-parsing 0.0.1.dev0
    Uninstalling tapas-table-parsing-0.0.1.dev0:
      Successfully uninstalled tapas-table-parsing-0.0.1.dev0
  Found existing installation: cachetools 4.1.1
    U

## Fetch Pretrained Models form Google

In [140]:
! gsutil cp gs://tapas_models/2020_04_21/tapas_sqa_base.zip . && unzip tapas_sqa_base.zip


Copying gs://tapas_models/2020_04_21/tapas_sqa_base.zip...
| [1 files][  1.0 GiB/  1.0 GiB]    6.1 MiB/s                                   
Operation completed over 1 objects/1.0 GiB.                                      
Archive:  tapas_sqa_base.zip
   creating: tapas_sqa_base/
  inflating: tapas_sqa_base/model.ckpt.data-00000-of-00001  
  inflating: tapas_sqa_base/model.ckpt.index  
  inflating: tapas_sqa_base/README.txt  
  inflating: tapas_sqa_base/vocab.txt  
  inflating: tapas_sqa_base/bert_config.json  
  inflating: tapas_sqa_base/model.ckpt.meta  


## Import

In [141]:
import tensorflow.compat.v1 as tf
import os
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

### Utilities

In [142]:

from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils


##Load checkpoint for prediction
The code is to create Interaction protobuf object.  interaction_pb2. which is the datastructure we use to store examples, and then call the prediction script.

In [143]:
os.makedirs('results/sqa/tf_examples', exist_ok=True)
os.makedirs('results/sqa/model', exist_ok=True)
with open('results/sqa/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_sqa_base/model.ckpt{suffix}', f'results/sqa/model/model.ckpt-0{suffix}')

Install HANA Driver

In [144]:
'''
!pip install git+https://github.com/SAP/pyhdb.git
#!pip install ./pyhdb
'''

'\n!pip install git+https://github.com/SAP/pyhdb.git\n#!pip install ./pyhdb\n'

In [145]:
'''
import pyhdb
import inspect
'''

'\nimport pyhdb\nimport inspect\n'

In [146]:
#inspect.signature(pyhdb.connect)

In [147]:
#Create a connection object

'''
connection = pyhdb.connect(
    host="bihanasbx.corp.service-now.com",
    # bihanadev.corp.service-now.com:30115
    port=30115,
    user="VMALIREDDY",
    password="F@kn21Od3#mqw"
    )
'''



'\nconnection = pyhdb.connect(\n    host="bihanasbx.corp.service-now.com",\n    # bihanadev.corp.service-now.com:30115\n    port=30115,\n    user="VMALIREDDY",\n    password="F@kn21Od3#mqw"\n    )\n'

In [148]:
'''
cursor = connection.cursor()
cursor.execute("select * from SAP_ECC_NOW.PURCHASE_TO_PAY.DataCenter.EnterpriseModel/DATACENTER_SPEND")
dc_data = cursor.fetchall()
'''

'\ncursor = connection.cursor()\ncursor.execute("select * from SAP_ECC_NOW.PURCHASE_TO_PAY.DataCenter.EnterpriseModel/DATACENTER_SPEND")\ndc_data = cursor.fetchall()\n'

In [149]:
# !pip install pyodbc

In [150]:
# Import libraries
#import pyodbc

In [151]:
# HANA Connection Object
#conn = pyodbc.connect('DSN=HanaODBC')

### Read pickle file

In [152]:
#!!dir downloads

In [153]:
#dc_df=pd.read_pickle("C:\\Users\\Venugopal.Malireddy\\Downloads\\cloud_spend.pkl")#
#C:\Users\Venugopal.Malireddy\Downloads

In [154]:
from google.colab import files


In [155]:
uploaded = files.upload()

Saving cloud_spend-1.pkl to cloud_spend-1.pkl


In [156]:
import io
df = pd.read_pickle(io.BytesIO(uploaded['cloud_spend-1.pkl']))
# Dataset is now stored in a Pandas Dataframe

In [157]:
df.shape 


(70390, 125)

In [158]:
list(df.columns.values)

['isHardware',
 'IsCabinet',
 'ParentVendorOld',
 'PR_Number',
 'PrDescription',
 'PRStateDescription',
 'PrStage',
 'GeoRegion',
 'Region',
 'VendorName',
 'PrType',
 'DepartmentName',
 'FunctionalArea',
 'ReportingYearPrCloseDate',
 'ReportingQuarterPrCloseDate',
 'ReportingMonthPrCloseDate',
 'YearPrCloseDate',
 'QuarterByYearPrCloseDate',
 'MonthByYearPrCloseDate',
 'MaterialGroup',
 'ModelCategory',
 'ModelType',
 'ModelShortDescription',
 'ModelNumber',
 'ModelName',
 'ManufacturerName',
 'PR_RevisionAmountInUSD_Char',
 'PrLineNumber',
 'PrLineItemShortDesc',
 'PrLineItemDesc',
 'PrLine_ATF_Flag_Val',
 'GLAccountInlineItem',
 'ReceivingPlantSysID',
 'PrCreateDate',
 'PrCloseDate',
 'PurchasingGroup',
 'CompanyCodeName',
 'CostCenter',
 'MonthPrCreateDate',
 'MonthByYearPrCreateDate',
 'QuarterByYearPrCreateDate',
 'YearPrCreateDate',
 'MonthPrCloseDate',
 'EXPENSE_ACCOUNT',
 'GlobalUltimateBusName',
 'Vendor',
 'GL_CodeDescription',
 'CompanyCodeRegion',
 'ReportingMonthPrCreateD

In [159]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [160]:
df.describe()

,ParentAsset,ChildAsset,PercentInvoiced,PercentDelivered,LineIDInPrLineItems,ShipmentRecocciled,PR_AmountInUSD,PR_AmountInUSD_LineItems,NetPriceInPrLineItems,ToBeDeliveredAmount,ToBeInvoicedAmount,DeliveredAmount,TotalAmountUsd,ReceivedQuantity,POInvoicedPercentage,PODeliveredPercentage,PO_Value,POQuantity,OPEX1
count,70390.000,70390.000,70289.000,70289.000,69655.000,30735.000,70390.000,70390.000,70390.000,47012.000,47012.000,1967.000,44579.000,40631.000,63433.000,58242.000,63433.000,63433.000,26907.000
mean,0.093,0.411,0.734,0.734,192.543,0.000,3228108.027,22391.854,8059.751,1370.386,5770.358,29042.895,1808280.762,18.287,53.846,91.370,1017575.369,448313.754,19663.712
std,0.291,0.492,8.333,8.368,223.719,0.000,5328759.541,130928.347,84102.266,71935.773,107500.765,68908.175,1641348.256,13.293,49.318,27.830,37127914.905,37118039.646,164854.537
min,0.000,0.000,0.000,0.000,10.000,0.000,1.000,0.010,0.010,0.000,0.000,0.000,949.450,0.000,0.000,0.000,0.000,0.160,0.145
25%,0.000,0.000,0.000,0.000,50.000,0.000,172999.230,126.320,31.590,0.000,0.000,0.000,658290.480,6.000,0.000,100.000,55300.550,77.000,256.590
50%,0.000,0.000,0.000,0.000,130.000,0.000,749460.900,900.000,364.730,0.000,0.000,0.000,1255764.690,17.000,99.860,100.000,414364.180,131.000,1105.600
75%,0.000,1.000,0.000,0.000,260.000,0.000,4129941.120,3934.791,2266.780,0.000,0.000,228.000,2240466.280,33.000,100.000,100.000,746610.120,297.000,6059.035
max,1.000,1.000,200.000,200.000,2440.000,0.000,27294644.160,9900000.000,15375100.000,8190720.000,8190720.000,346563.000,9756857.460,111.000,118.520,100.000,4119974850.000,4119974850.000,6521197.114


In [161]:
# dc_df = dc_df[dc_df[]]

In [162]:
'''
dc_df = dc_df[['PR_Number','PrCreateDate','PrDescription','PRStateDescription','VendorName','DepartmentName','FunctionalArea',
              'SAP_PO_Number','PO_CreatedDate','ExpType',
              'ReceivingSlip','RecevingSlipStatus','RS_ReceivingDate','ShipDate','VendorShipmentId',
              'POLeadTime','Late','PR_AmountInUSD','ToBeDeliveredAmount']]
'''
dc_df = df[['PR_Number','PrCreateDate','PRStateDescription','VendorName',
              'ExpType',
              'RecevingSlipStatus','RS_ReceivingDate',
              'POLeadTime','Late','PR_AmountInUSD']]

In [163]:
dc_df.shape


(70390, 10)

In [164]:
dc_df.head()

,PR_Number,PrCreateDate,PRStateDescription,VendorName,ExpType,RecevingSlipStatus,RS_ReceivingDate,POLeadTime,Late,PR_AmountInUSD
0,PR0095868,2020-01-13,Closed Complete,SHI International Corp.,CAPEX,Complete,2020-07-17,186,-88,1431151.850
1,PR0102699,2020-05-18,Closed Complete,"Redapt Holdings, Inc. Redapt, Inc.",CAPEX,Complete,2020-07-17,60,-7,16210.520
2,PR0102699,2020-05-18,Closed Complete,"Redapt Holdings, Inc. Redapt, Inc.",OPEX,Complete,2020-07-17,60,-7,16210.520
3,PR0102900,2020-05-21,Closed Complete,"Redapt Holdings, Inc. Redapt, Inc.",CAPEX,Complete,2020-07-17,57,-17,89326.280
4,PR0102900,2020-05-21,Closed Complete,"Redapt Holdings, Inc. Redapt, Inc.",OPEX,Complete,2020-07-17,57,-17,89326.280


In [165]:
dc_df.columns

Index(['PR_Number', 'PrCreateDate', 'PRStateDescription', 'VendorName',
       'ExpType', 'RecevingSlipStatus', 'RS_ReceivingDate', 'POLeadTime',
       'Late', 'PR_AmountInUSD'],
      dtype='object')

In [166]:
dc_df=dc_df[:20]

In [167]:
dc_df.columns.size

10

In [168]:
dc_df = dc_df.astype(str)

In [169]:
list_of_list=[[]]
list_of_list[0] = list(dc_df.columns)
list_of_list.extend(dc_df.values.tolist())
#list_of_list[[]]

### Helper functions

In [170]:
max_seq_length =512
vocab_file = "tapas_sqa_base/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries):
  table = table_data
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/sqa/tf_examples/test.tfrecord", examples)
  write_tf_example("results/sqa/tf_examples/random-split-1-dev.tfrecord", [])
  
  ! python tapas/tapas/run_task_main.py \
    --task="SQA" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --init_checkpoint="tapas_sqa_base/model.ckpt" \
    --bert_config_file="tapas_sqa_base/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/sqa/model/test_sequence.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  print()
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
      coordinates = prediction_utils.parse_coordinates(row["answer_coordinates"])
      all_coordinates.append(coordinates)
      answers = ', '.join([table[row + 1][col] for row, col in coordinates])
      position = int(row['position'])
      print(">", queries[position])
      print(answers)
  return all_coordinates

In [171]:
!ls -l

total 1243820
-rw-r--r-- 1 root root  104114760 Oct 30 00:46 cloud_spend-1.pkl
-rw-r--r-- 1 root root       4338 Oct 30 00:21 ConverseQuery.py
-rwxr-xr-x 1 root root   26683198 Oct  8  2019 ngrok
-rw-r--r-- 1 root root   13773305 Oct 29 18:36 ngrok-stable-linux-amd64.zip
-rw-r--r-- 1 root root   13773305 Oct 29 19:17 ngrok-stable-linux-amd64.zip.1
drwxr-xr-x 3 root root       4096 Oct 30 00:26 results
drwxr-xr-x 4 root root       4096 Oct 29 16:23 sqa
drwxr-xr-x 5 root root       4096 Oct 30 00:25 tapas
drwxr-x--- 2 root root       4096 Apr 29  2020 tapas_sqa_base
-rw-r--r-- 1 root root 1115287043 Oct 30 00:26 tapas_sqa_base.zip


In [172]:
dc_df.shape

(20, 10)

In [173]:
list_of_list[:5]

[['PR_Number',
  'PrCreateDate',
  'PRStateDescription',
  'VendorName',
  'ExpType',
  'RecevingSlipStatus',
  'RS_ReceivingDate',
  'POLeadTime',
  'Late',
  'PR_AmountInUSD'],
 ['PR0095868',
  '2020-01-13',
  'Closed Complete',
  'SHI International Corp.',
  'CAPEX',
  'Complete',
  '2020-07-17',
  '186',
  '-88',
  '1431151.85'],
 ['PR0102699',
  '2020-05-18',
  'Closed Complete',
  'Redapt Holdings, Inc. Redapt, Inc.',
  'CAPEX',
  'Complete',
  '2020-07-17',
  '60',
  '-7',
  '16210.52'],
 ['PR0102699',
  '2020-05-18',
  'Closed Complete',
  'Redapt Holdings, Inc. Redapt, Inc.',
  'OPEX',
  'Complete',
  '2020-07-17',
  '60',
  '-7',
  '16210.52'],
 ['PR0102900',
  '2020-05-21',
  'Closed Complete',
  'Redapt Holdings, Inc. Redapt, Inc.',
  'CAPEX',
  'Complete',
  '2020-07-17',
  '57',
  '-17',
  '89326.28']]

## Predict from Cloud Spend 

In [174]:
result = predict(list_of_list, ["What is the maximum POLeadTime?",
                                "Show me all Vendor Names?",
                                "What is total PR amount for CAPEX?",
                                "Show PR number with Closed Complete status?"])

   
     # ,
      #"Show me  PR Status that are not Closed Complete?",
      #"What is PR_AmountInUSD ExpenseType ?"])

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


PR_Number,PrCreateDate,PRStateDescription,VendorName,ExpType,RecevingSlipStatus,RS_ReceivingDate,POLeadTime,Late,PR_AmountInUSD
PR0095868,2020-01-13,Closed Complete,SHI International Corp.,CAPEX,Complete,2020-07-17,186,-88,1431151.85
PR0102699,2020-05-18,Closed Complete,"Redapt Holdings, Inc. Redapt, Inc.",CAPEX,Complete,2020-07-17,60,-7,16210.52
PR0102699,2020-05-18,Closed Complete,"Redapt Holdings, Inc. Redapt, Inc.",OPEX,Complete,2020-07-17,60,-7,16210.52
PR0102900,2020-05-21,Closed Complete,"Redapt Holdings, Inc. Redapt, Inc.",CAPEX,Complete,2020-07-17,57,-17,89326.28
PR0102900,2020-05-21,Closed Complete,"Redapt Holdings, Inc. Redapt, Inc.",OPEX,Complete,2020-07-17,57,-17,89326.28
PR0102268,2020-05-06,Closed Complete,"Redapt Holdings, Inc. Redapt, Inc.",OPEX,nan,nan,nan,nan,824.42
PR0104990,2020-07-08,Closed Complete,SHI GLOBAL IT SOLUTIONS IRELAND LIMITED,OPEX,nan,nan,nan,nan,16544.74
PR0104990,2020-07-08,Closed Complete,SHI GLOBAL IT SOLUTIONS IRELAND LIMITED,OPEX,nan,nan,nan,nan,16544.74
PR0104990,2020-07-08,Closed Complete,SHI GLOBAL IT SOLUTIONS IRELAND LIMITED,OPEX,nan,nan,nan,nan,16544.74
PR0104990,2020-07-08,Closed Complete,SHI GLOBAL IT SOLUTIONS IRELAND LIMITED,OPEX,nan,nan,nan,nan,16544.74



> What is the maximum POLeadTime?
186
> Show me all Vendor Names?
SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, Redapt Holdings, Inc. Redapt, Inc., SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, Redapt Holdings, Inc. Redapt, Inc., SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, Redapt Holdings, Inc. Redapt, Inc., SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, Redapt Holdings, Inc. Redapt, Inc., SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI International Corp., SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, Redapt Holdings, Inc. Redapt, Inc.
> What is total PR amount for CAPEX?
SHI International Corp., Redapt Holdings, Inc. Redapt, Inc., Redapt Holdings, Inc. Redapt, Inc.
> Show PR number with 

In [104]:
print(result)

[{(0, 7)}, {(7, 3), (8, 3), (1, 3), (18, 3), (3, 3), (13, 3), (15, 3), (16, 3), (9, 3), (11, 3), (6, 3), (17, 3), (2, 3), (19, 3), (4, 3), (14, 3), (0, 3), (10, 3), (12, 3), (5, 3)}, {(0, 3), (1, 3), (3, 3)}, set()]


### Download Streamlit

In [176]:
!pip install streamlit -q

In [178]:
import streamlit as st

ContextualVersionConflict: ignored

## Create A Streamlit application



In [179]:
#streamlit run foo.py

In [180]:
!ls -l

total 1243824
-rw-r--r-- 1 root root  104114760 Oct 30 00:46 cloud_spend-1.pkl
-rw-r--r-- 1 root root       4338 Oct 30 00:21 ConverseQuery.py
-rw-r--r-- 1 root root       2557 Oct 30 00:48 error
-rwxr-xr-x 1 root root   26683198 Oct  8  2019 ngrok
-rw-r--r-- 1 root root   13773305 Oct 29 18:36 ngrok-stable-linux-amd64.zip
-rw-r--r-- 1 root root   13773305 Oct 29 19:17 ngrok-stable-linux-amd64.zip.1
drwxr-xr-x 3 root root       4096 Oct 30 00:26 results
drwxr-xr-x 4 root root       4096 Oct 29 16:23 sqa
drwxr-xr-x 5 root root       4096 Oct 30 00:25 tapas
drwxr-x--- 2 root root       4096 Apr 29  2020 tapas_sqa_base
-rw-r--r-- 1 root root 1115287043 Oct 30 00:26 tapas_sqa_base.zip


In [181]:
!ls -l  content

ls: cannot access 'content': No such file or directory


In [205]:
%%writefile ConverseQuery.py
import streamlit as st
PAGE_CONFIG = {"page_title":"StColab.io","page_icon":":smiley:","layout":"centered"}
st.set_page_config(**PAGE_CONFIG)

#---------------------------------------------------------

#---------------------------------------------------------
def main():
  
  st.title("Conversational Query - ML App")
  st.header("   Google - BERT Based")
  st. write(" The answers are :")

  menu = ["Home","About"]
  choice = st.sidebar.selectbox('Menu',menu)
  if choice == 'Home':
    st.sidebar.subheader(" User Input")
    st.sidebar.subheader("Model: DATACENTER_SPEND_ANALYSIS")
    st.sidebar.write("-------------------------------------")
    user_input1 = st.sidebar.text_input("What is the maximum POLeadTime?")
    user_input2 = st.sidebar.text_input("Show me all Vendor Names?")
    user_input3 = st.sidebar.text_input("What is total PR amount for CAPEX? ")
    ExecuteQury = st.sidebar.button("Execute Query")
    #queryResults = predict(list_of_list,["What is the maximum POLeadTime?","Show me all Vendor Names?"])
    st.subheader("Results from query :")
    #st.subheader(queryResults)

    #if ExecuteQury:
      #queryResults1 = predict(list_of_list,
      #                       ["What is the maximum POLeadTime?",
      #                        "Show me all Vendor Names?"])
			#st.write(predict(list_of_list,
			#        ["What is the maximum POLeadTime?",
			#				     "Show me all Vendor Names?"]))
	    #st.subheader(queryResults1)

    #st.sidebar.subheader("SAP_ECC_NOW.PURCHASE_TO_PAY.DataCenter.ReportingViews/DATACENTER_SPEND_ANALYSIS:")


if __name__ == '__main__':
	main()

Overwriting ConverseQuery.py


In [183]:
'''
result = predict(list_of_list, ["What is the maximum POLeadTime?",
                                "Show me all Vendor Names?",
                                "What is total PR amount for CAPEX?",
                                "Show PR number with Closed Complete status?"])

   
     # ,
      #"Show me  PR Status that are not Closed Complete?",
      #"What is PR_AmountInUSD ExpenseType ?"])
'''

'\nresult = predict(list_of_list, ["What is the maximum POLeadTime?",\n                                "Show me all Vendor Names?",\n                                "What is total PR amount for CAPEX?",\n                                "Show PR number with Closed Complete status?"])\n\n   \n     # ,\n      #"Show me  PR Status that are not Closed Complete?",\n      #"What is PR_AmountInUSD ExpenseType ?"])\n'

In [199]:

result = predict(list_of_list, ["What is the maximum POLeadTime?",
                                "Show me all Vendor Names?",
                                "What is total PR amount for CAPEX?",
                                "Show PR number with Closed Complete status?"])

   
     # ,
      #"Show me  PR Status that are not Closed Complete?",
      #"What is PR_AmountInUSD ExpenseType ?"])


is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


PR_Number,PrCreateDate,PRStateDescription,VendorName,ExpType,RecevingSlipStatus,RS_ReceivingDate,POLeadTime,Late,PR_AmountInUSD
PR0095868,2020-01-13,Closed Complete,SHI International Corp.,CAPEX,Complete,2020-07-17,186,-88,1431151.85
PR0102699,2020-05-18,Closed Complete,"Redapt Holdings, Inc. Redapt, Inc.",CAPEX,Complete,2020-07-17,60,-7,16210.52
PR0102699,2020-05-18,Closed Complete,"Redapt Holdings, Inc. Redapt, Inc.",OPEX,Complete,2020-07-17,60,-7,16210.52
PR0102900,2020-05-21,Closed Complete,"Redapt Holdings, Inc. Redapt, Inc.",CAPEX,Complete,2020-07-17,57,-17,89326.28
PR0102900,2020-05-21,Closed Complete,"Redapt Holdings, Inc. Redapt, Inc.",OPEX,Complete,2020-07-17,57,-17,89326.28
PR0102268,2020-05-06,Closed Complete,"Redapt Holdings, Inc. Redapt, Inc.",OPEX,nan,nan,nan,nan,824.42
PR0104990,2020-07-08,Closed Complete,SHI GLOBAL IT SOLUTIONS IRELAND LIMITED,OPEX,nan,nan,nan,nan,16544.74
PR0104990,2020-07-08,Closed Complete,SHI GLOBAL IT SOLUTIONS IRELAND LIMITED,OPEX,nan,nan,nan,nan,16544.74
PR0104990,2020-07-08,Closed Complete,SHI GLOBAL IT SOLUTIONS IRELAND LIMITED,OPEX,nan,nan,nan,nan,16544.74
PR0104990,2020-07-08,Closed Complete,SHI GLOBAL IT SOLUTIONS IRELAND LIMITED,OPEX,nan,nan,nan,nan,16544.74



> What is the maximum POLeadTime?
186
> Show me all Vendor Names?
SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, Redapt Holdings, Inc. Redapt, Inc., SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, Redapt Holdings, Inc. Redapt, Inc., SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, Redapt Holdings, Inc. Redapt, Inc., SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, Redapt Holdings, Inc. Redapt, Inc., SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI International Corp., SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, SHI GLOBAL IT SOLUTIONS IRELAND LIMITED, Redapt Holdings, Inc. Redapt, Inc.
> What is total PR amount for CAPEX?
SHI International Corp., Redapt Holdings, Inc. Redapt, Inc., Redapt Holdings, Inc. Redapt, Inc.
> Show PR number with 

### Check File created

In [72]:
!ls -l

total 8
-rw-r--r-- 1 root root 1496 Oct 29 18:21 ConverseQuery.py
drwxr-xr-x 4 root root 4096 Oct 29 16:23 sqa


### Install ngrok

In [185]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

--2020-10-30 00:54:33--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.1.26.21, 52.0.235.222, 54.236.74.205, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.1.26.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.13M  5.95MB/s    in 2.2s    

2020-10-30 00:54:36 (5.95 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13773305/13773305]

replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [186]:
!streamlit run ConverseQuery.py &>/dev/null&

In [187]:
get_ipython().system_raw('./ngrok http 8501 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
"import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://84041b267d22.ngrok.io


In [98]:

#!streamlit run ConverseQuery.py

In [ ]:
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

In [ ]:
#!unzip ngrok-stable-linux-amd64.zip

In [ ]:
#! pip install pyngrok

  Created wheel for pyngrok: filename=pyngrok-5.0.0-cp36-none-any.whl size=18780 sha256=7cd1d4a22cc9dffce047e5df1ec584242da8dbfbd06513a83129c5ad1fbf850a
  Stored in directory: /root/.cache/pip/wheels/95/df/23/af8dde08c3fcdc7b966adcacef48ab29aa3b0b1860df5d2b79
Successfully built pyngrok


In [29]:
!ngrok

/bin/bash: ngrok: command not found


In [106]:
!pgrep ngrok

1997
2031


In [108]:
!kill 2031

In [13]:
!pgrep streamlit

In [ ]:
!ngrok authtoken 1jLjJnrh3EQPYjoUdjH9IBzWa66_5GC8hxEPV3r9HKjZvHEQN

/bin/bash: ngrok: command not found


In [ ]:
from pyngrok import ngrok

### Run app in background

In [118]:
!ls -l


total 52972
-rw-r--r-- 1 root root     1537 Oct 29 21:24 ConverseQuery.py
-rwxr-xr-x 1 root root 26683198 Oct  8  2019 ngrok
-rw-r--r-- 1 root root 13773305 Oct 29 18:36 ngrok-stable-linux-amd64.zip
-rw-r--r-- 1 root root 13773305 Oct 29 19:17 ngrok-stable-linux-amd64.zip.1
drwxr-xr-x 4 root root     4096 Oct 29 16:23 sqa


In [120]:
%cd content

[Errno 2] No such file or directory: 'content'
/content/results


In [ ]:
!streamlit

In [ ]:
!streamlit run ConverseQuery.py &>/dev/null&

### check if streamlit is running


In [17]:
!pgrep streamlit


In [ ]:
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')


In [ ]:
public_url


### END


In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

In [ ]:
#!streamlit run ConverseQuery.py &>/dev/null&

In [ ]:
!streamlit run ConverseQuery.py

# --- END--

In [ ]:
!pip install pyngrok

  Created wheel for pyngrok: filename=pyngrok-5.0.0-cp36-none-any.whl size=18780 sha256=e5a8c60259a523774f2edeb60a5db85905fa5baf3105f98bdb3f88b3fc1aee0e
  Stored in directory: /root/.cache/pip/wheels/95/df/23/af8dde08c3fcdc7b966adcacef48ab29aa3b0b1860df5d2b79
Successfully built pyngrok


In [ ]:
from pyngrok import ngrok


In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')


PyngrokNgrokHTTPError: ignored

In [ ]:
public_url

<NgrokTunnel: "http://366e6364ef36.ngrok.io" -> "http://localhost:80">

In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://15effef262a5.ngrok.io


### Run strealit and send it to backgroubd

In [ ]:
#!nohub streamlit run ConverseQuery.py &

In [ ]:
!streamlit run ConverseQuery.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.201.230.130:8501



In [ ]:
!streamlit run ConverseQuery.py &>/dev/null&

In [ ]:
!streamlit run ConverseQuery.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8502
  External URL: http://35.201.230.130:8502

  Stopping...


## Run above URL

In [ ]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501

public_url = ngrok.connect(port='8501')
public_url

ModuleNotFoundError: ignored

### Craete App 

In [ ]:
# list ./content/ConverseQuery.py

In [ ]:
# ngrok.kill()

## Run strealit and send it to backgroubd

In [ ]:
!streamlit run ConverseQuery.py &>/dev/null&

In [ ]:
#!nohub streamlit run ConverseQuery.py &

### check if it is running

In [ ]:
!pgrep streamlit

In [ ]:
# purl = ngrok.connect(port='8501')

In [ ]:
#purl

### use secure tunnel


In [ ]:
!ngrok

In [ ]:
!pgrep streamlit